# PROJECT - API WEB SCRAPPING

## 1.- API 

## Retrieve set cards from API

In [1]:
#import libraries

import json
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time

In [2]:
# extract sets from scryfall API

complete = []
url = 'https://api.scryfall.com/sets'
response = requests.get(url)
results = response.json()
x = json_normalize(results)
flattened_data1 = json_normalize(x.data[0])
complete.append(flattened_data1)
scryfall = pd.concat(complete, sort=False)

In [3]:
scryfall.head()

,object,id,code,name,uri,scryfall_uri,search_uri,released_at,set_type,card_count,parent_set_code,digital,foil_only,icon_svg_uri,tcgplayer_id,mtgo_code,arena_code,block_code,block
0,set,8fe3f935-7c8d-4a4e-a051-c0b0f251d262,tund,Unsanctioned Tokens,https://api.scryfall.com/sets/8fe3f935-7c8d-4a...,https://scryfall.com/sets/tund,https://api.scryfall.com/cards/search?order=se...,2020-02-29,token,6,und,False,False,https://img.scryfall.com/sets/default.svg?1581...,NaN,NaN,NaN,NaN,NaN
1,set,fccfdf97-f5f2-43b4-9be9-9255414e6633,und,Unsanctioned,https://api.scryfall.com/sets/fccfdf97-f5f2-43...,https://scryfall.com/sets/und,https://api.scryfall.com/cards/search?order=se...,2020-02-29,funny,96,NaN,False,False,https://img.scryfall.com/sets/default.svg?1581...,2598.0,NaN,NaN,NaN,NaN
2,set,66d787e4-101d-4f72-a4ed-7c38df9b99fe,pthb,Theros Beyond Death Promos,https://api.scryfall.com/sets/66d787e4-101d-4f...,https://scryfall.com/sets/pthb,https://api.scryfall.com/cards/search?order=se...,2020-01-24,promo,136,thb,False,False,https://img.scryfall.com/sets/thb.svg?1581915600,NaN,NaN,NaN,NaN,NaN
3,set,200c397b-bf57-46a2-8ebf-592148fd49a4,tthb,Theros Beyond Death Tokens,https://api.scryfall.com/sets/200c397b-bf57-46...,https://scryfall.com/sets/tthb,https://api.scryfall.com/cards/search?order=se...,2020-01-24,token,14,thb,False,False,https://img.scryfall.com/sets/thb.svg?1581915600,NaN,NaN,NaN,NaN,NaN
4,set,5f23a78d-cda1-462a-8be3-a62b40c34913,thb,Theros Beyond Death,https://api.scryfall.com/sets/5f23a78d-cda1-46...,https://scryfall.com/sets/thb,https://api.scryfall.com/cards/search?order=se...,2020-01-24,expansion,358,NaN,False,False,https://img.scryfall.com/sets/thb.svg?1581915600,2568.0,thb,thb,NaN,NaN


In [4]:
#extract the legal core and expansion sets which are from Return to Ravnica set to actual
core = scryfall[scryfall['set_type'] == 'core']
expansion = scryfall[scryfall['set_type'] == 'expansion']
set_list = pd.concat([core, expansion])
set_list = set_list.filter(['code', 'name', 'released_at', 'block', 'card_count'])

# the initial sets include from 2012-10-05 (Return to Ravnica set) to actual set
pioneer_sets = set_list[set_list['released_at'] >= '2012-10-05']

#append 'Welcome Decks' which are also legal
pioneer_sets = pd.concat([pioneer_sets, set_list.filter(like='Welcome Deck', axis=0)])
pioneer_sets = pioneer_sets.sort_values(by=['released_at'])
pioneer_sets['code'] = pioneer_sets['code'].apply(lambda x: x.upper())
pioneer_sets = pioneer_sets.set_index('code',inplace=False).fillna('Unknown')
pioneer_sets.columns = ['Set Name', 'Release Date', 'Block', 'Total Cards']
pioneer_sets['Total Cards'].sum()

7795

## Retrieve the list of all cards from the legal sets

In [5]:
# Retrieve the number of pages needed for request
total_cards_set = pioneer_sets['Total Cards'].to_list()
page_numbers = [i // 100 + 1 for i in total_cards_set]

complete = []
for i in pioneer_sets.index:
    for j in range(1,page_numbers[0]+1):
        url='https://api.magicthegathering.io/v1/cards?page='+str(j)+'&set='+str(i)
        response = requests.get(url)
        time.sleep(0.05)
        results = response.json()
        x = json_normalize(results)
        flattened_data1 = json_normalize(x.cards[0])
        complete.append(flattened_data1)
        
data = pd.concat(complete, sort=False, ignore_index=True)


In [6]:
# Select only columns with valuable info
cards = data[['name', 'manaCost', 'cmc', 'colors', 'type', 'rarity', 'set', 'text', 'power', 'toughness', 'loyalty']]

In [7]:
# Extract the list of banned cards

url = 'https://magic.wizards.com/es/game-info/gameplay/formats/pioneer'

def get_links(url):
    '''
    This function takes a url of a MTG format and
    scrappes the list of the cards that are banned
    '''
    html = requests.get(url).content
    soup = BeautifulSoup(html, features='html.parser')
    links = soup.find_all('a', {'class':'autocard-link'})
    
    # from each link take the 2 second last 
    # (which contains the name of the card)
    # and strip the last character that are related with
    
    reference = [str(i).split('>')[-2][:-3] for i in links]
    return reference

banned_cards = get_links(url)

In [8]:
print(banned_cards)

['Bloodstained Mire', 'Felidar Guardian', 'Field of the Dead', 'Flooded Strand', 'Leyline of Abundance', 'Nexus of Fate', 'Oath of Nissa', 'Oko, Thief of Crowns', 'Once Upon a Time', 'Polluted Delta', 'Smuggler’s Copter', 'Veil of Summer', 'Windswept Heath ', 'Wooded Foothills']


## Extract the matches results from the last competitive tournaments 

In [9]:
def get_matches(city):
    '''
    This function takes the name of a city an raises an error if the city is not valid.
    In case the city if one of the cities (currently brussels, nagoya and phoenix) that had a tournament, 
    creates a dataframe with all the different matches.
    '''
    tabs = []
    # check the city before passing to the url
    if city.lower() == 'brussels':
        range_max = 17
    elif (city.lower() == 'nagoya') | (city.lower() == 'phoenix'):
        range_max = 16
    else:
        raise ValueError('The city provided is not valid')
        
    for i in range(1,range_max):
        url = 'https://magic.gg/news/players-tour-'+str(city)+'-2020-round-'+str(i)+'-results'
        html = requests.get(url).content
        soup = BeautifulSoup(html, features='html.parser')
        table = soup.find_all('div', {'class': 'table-wrapper'})[0].find_all('tr')
    
        for tr in table:
            tabs.append({
                'Player': tr.find_all('td')[1].string.rstrip().title(),
                'Opponent': tr.find_all('td')[5].string.title(),
                'Match': tr.find_all('td')[3].string
                })
    return pd.DataFrame(tabs)

In [10]:
brussels_matches = get_matches('brussels')
brussels_matches.head()

,Player,Opponent,Match
0,"Caparroz, Joao Lucas","Gauthier, Martin-Eric",Won 2-0
1,"Knittel, Fabrice","Calafell, Joel",Won 2-0
2,"Oneto, Mattia","Choca, Joao",Lost 0-2
3,"Jao, Jeff","Pereira, Alexandre",Won 2-1
4,"Wijsman, Gerwin","Rodrigues, Thiago",Lost 1-2


In [11]:
# nagoya matches
nagoya_matches = get_matches('nagoya')
nagoya_matches.head()

,Player,Opponent,Match
0,"Collins, Michael","Nishimoto, Sosuke",Won 2-1
1,"Nakamura, Shuhei","Masahide, Moriyama",Won 2-0
2,"Sloo, Felix","Lip, Quinton",Lost 1-2
3,"Huang, Hao-Shan","Sze Hang, Chan",Lost 1-2
4,"Kabashima, Takuya","Boku, Takashi",Lost 1-2


In [12]:
# phoenix matches
phoenix_matches = get_matches('phoenix')
phoenix_matches.head()

,Player,Opponent,Match
0,"Whitehouse, Evan","Van Vaals, Michael",Won 2-0
1,"Mechtensimer, Aaron","Donegan, Dylan",Lost 1-2
2,"Luong, Marcus","Scott-Vargas, Luis",Lost 0-2
3,"Bell, Carson","Norton, Tony",Won 2-0
4,"Henry, Julien","An, Hui Jae",Won 2-0


## Extract the decklist the different player at each tournament

In [13]:
def get_decklist(city):
    '''
    This function takes the name of a city an raises an error if the city is not valid.
    In case the city if one of the cities (currently brussels, nagoya and phoenix) that had a tournament, 
    creates a dataframe with all the decklist played by the different players.
    
    After extracting data, it makes a transformation because card frequency and card list are extracted as lists. 
    Therefore, for each card, it creates a new column and stores its number of copies in each deck.
    '''
    # check if the city is correct
    
    if city.lower() == 'nagoya':
        range_max = 4
    elif (city.lower() == 'brussels') | (city.lower() == 'phoenix'):
        range_max = 6
    else:
        raise ValueError('The city provided is not valid')
        
    card_list = []
    card_count = []
    span_cards = []
    player_deck = []
    
    for s in range(1,range_max):
        # extract the data from web
        url = 'https://magic.wizards.com/en/events/coverage/players-tour-'+str(city)+'-2020-decklists-'+str(s)
        html = requests.get(url).content
        soup = BeautifulSoup(html, features='html.parser')
        player = [str(title.text) for title in soup.find_all('h4')]
        # append the results
        player_deck.append(player)
       
        for deck in soup.find_all('div', {'class':'deck-list-text'}):
            a_tag = deck.find_all('span', {'class':'card-name'})
            count_tag = deck.find_all('span', {'class':'card-count'})
            name = [str(tag).split('>')[-3][:-3] for tag in a_tag]
            count = [tag.text for tag in count_tag]
        
        # change non cards 'span class'
            span = [str(tag).split('>')[-2][:-6] for tag in a_tag]
    
        #append the results
            card_list.append(name)
            card_count.append(count)
            span_cards.append(span)

    for i, deck in enumerate(card_list):
        for j, card in enumerate(deck):
            if '<span class' in card:
                card_list[i][j] = span_cards[i][j]


    player_list = [(i.split(',')[0] + str(',') + i.split(',')[1].split('-')[0].rstrip()).title() 
                   for page in player_deck for i in page]
    deck_list = ['-'.join(i.split(',')[1].split('-')[1:]) for page in player_deck for i in page]


    decklist = pd.DataFrame([player_list, deck_list, card_count, card_list], index=['Player', 'Deck', 'Copies', 'Cards'])
    decklist = decklist.T
    
    ### Extract all the cards stored in card list as new columns in the dataframe and assign card count as values ###
    
    #Dict comprehension with key = card name, value = card count
    card_frequency = [dict(zip(card_list[i], card_count[i])) for i in range(len(card_list))]

    card_values =  pd.DataFrame([card_frequency], index=['cards'])

    # extract the dict values and create one column for each card
    card_values = card_values.T.cards.apply(pd.Series).fillna(0) 

    #reorder the columns alphabetically
    card_values = card_values.reindex(sorted(card_values.columns), axis=1)
    
    decklist = pd.concat([decklist.Player, decklist.Deck, card_values], axis=1, sort=False)
    
    return decklist

        

In [14]:
brussels_decklist = get_decklist('brussels')
brussels_decklist.head()

,Player,Deck,Abbot of Keral Keep,Abrade,Abrupt Decay,Absorb,Adanto Vanguard,Aether Gust,Aether Hub,Aetherflux Reservoir,...,Wizard's Retort,Woe Strider,Wolfwillow Haven,Woodland Cemetery,World Breaker,Yavimaya Coast,"Yorvo, Lord of Garenbrig",Young Pyromancer,"Zetalpa, Primal Dawn",Zurgo Bellstriker
0,"Adrien, Houssard",Mono-Black Aggro,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Airaksinen, Mikko",Dimir Inverter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Alawadhi, Abdullah",Dimir Inverter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albertini, Leonardo",Dimir Inverter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Alberyd, Johannes",Big Red,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
nagoya_decklist = get_decklist('nagoya')
nagoya_decklist.head()

,Player,Deck,Abbot of Keral Keep,Abrade,Abrupt Decay,Absorb,Adanto Vanguard,Aether Gust,Aether Hub,Aethersphere Harvester,...,Witchstalker,Wizard's Retort,Woe Strider,Woodland Cemetery,World Breaker,"Xenagos, the Reveler",Yarok's Fenlurker,Yavimaya Coast,Young Pyromancer,Zulaport Cutthroat
0,"Anan, Kazuyoshi",Niv to Light,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Asahara, Akira",Blue Black Inverter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Asano, Kurenai",Mono Black Aggro,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,"B.Abella, Lexter",Big Red,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bai, Hualin",Blue White Control,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
phoenix_decklist = get_decklist('phoenix')
phoenix_decklist.head()

,Player,Deck,Abbot of Keral Keep,Abrade,Abrupt Decay,Absorb,Adanto Vanguard,Aether Gust,Aether Hub,Aethersphere Harvester,...,Witching Well,Wizard's Lightning,Wizard's Retort,Wolfwillow Haven,Woodland Cemetery,World Breaker,Yahenni's Expertise,Yavimaya Coast,Zhur-Taa Goblin,Zurgo Bellstriker
0,"Abounorinejad, Faraz",Jund Delirium,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Adams, Evan",Mono-White Heliod,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Ahn, Sung",Jin - Bant Spirits,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Allen, Nolan",Bant Spirits,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"An, Hui Jae",Mono-Red Aggro,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Given the decklists dataframe, calculate the most played card (number of copies) for each tournament

In [17]:
def most_played_cards(decklist):
    '''
    This function takes a decklist type dataframe 
    and returns the most played card of this tournament
    '''
    # extract the name of all cards played
    card_columns = decklist.columns[2:]
    
    # data type
    decklist[card_columns] = decklist[card_columns].astype('int32')

    return decklist[card_columns].apply(np.sum, axis=0).sort_values(ascending=False)

In [18]:
brussels_played = most_played_cards(brussels_decklist)
brussels_played

Swamp                     1117
Island                     838
Plains                     590
Thoughtseize               554
Fatal Push                 512
                          ... 
Den Protector                1
Deputy of Detention          1
Nylea, Keen-Eyed             1
Nylea, God of the Hunt       1
Knight of Grace              1
Length: 720, dtype: int64

In [19]:
nagoya_played = most_played_cards(nagoya_decklist)
nagoya_played

Swamp                         527
Island                        436
Thoughtseize                  285
Fatal Push                    275
Plains                        253
                             ... 
Jace, Architect of Thought      1
Status // Statue                1
Primal Amulet                   1
Jace, Cunning Castaway          1
Remorseful Cleric               1
Length: 585, dtype: int64

In [20]:
phoenix_played = most_played_cards(phoenix_decklist)
phoenix_played

Swamp                        811
Island                       736
Thoughtseize                 663
Fatal Push                   642
Mountain                     636
                            ... 
Tocatli Honor Guard            1
Silkwrap                       1
Chemister's Insight            1
Tibalt, Rakish Instigator      1
Sunscourge Champion            1
Length: 569, dtype: int64

### Append the results to the total Cards Legal retrieved from API

In [21]:
# retrieve the list of the most played cards

total_played = pd.concat([brussels_played, nagoya_played, phoenix_played], axis=1, sort=True)
total_played.columns = ['Brussels', 'Nagoya', 'Phoenix']
total_played = total_played.fillna(0)
total_played = total_played.astype('int32')
total_played['Total copies'] = total_played.sum(axis=1, skipna = True)
total_played.sort_values(by='Total copies', ascending=False)
total_played = total_played.reset_index()
total_played.columns = ['Name', 'Brussels', 'Nagoya', 'Phoenix', 'Total copies']
total_played

,Name,Brussels,Nagoya,Phoenix,Total copies
0,Abbot of Keral Keep,23,28,65,116
1,Abrade,31,24,25,80
2,Abrupt Decay,107,48,112,267
3,Absorb,80,38,32,150
4,Adanto Vanguard,3,4,3,10
...,...,...,...,...,...
882,Young Pyromancer,18,8,0,26
883,"Zetalpa, Primal Dawn",24,0,0,24
884,Zhur-Taa Goblin,0,0,4,4
885,Zulaport Cutthroat,0,4,0,4


In [22]:
cards = data[['name', 'manaCost', 'cmc', 'colors', 'type', 'rarity', 'set', 'text', 'power', 'toughness', 'loyalty']]

In [23]:
cards = pd.merge(cards, total_played, left_on='name', right_on='Name')
cards.columns

Index(['name', 'manaCost', 'cmc', 'colors', 'type', 'rarity', 'set', 'text',
       'power', 'toughness', 'loyalty', 'Name', 'Brussels', 'Nagoya',
       'Phoenix', 'Total copies'],
      dtype='object')

In [24]:
cards = cards.drop('Name', axis=1)
cards.columns

Index(['name', 'manaCost', 'cmc', 'colors', 'type', 'rarity', 'set', 'text',
       'power', 'toughness', 'loyalty', 'Brussels', 'Nagoya', 'Phoenix',
       'Total copies'],
      dtype='object')

In [25]:
cards.columns = ['Name', 'Mana Cost', 'CMC', 'Colors', 'Type', 'Rarity', 'Set', 
                'Text', 'Power', 'Toughness', 'Loyalty', 'Brussels', 'Nagoya', 'Phoenix', 'Total']

cards.Power = cards.Power.fillna('Not a Creature')
cards.Toughness = cards.Toughness.fillna('Not a Creature')
cards.Loyalty = cards.Loyalty.fillna('Not a Planeswalker')

cards.head()

,Name,Mana Cost,CMC,Colors,Type,Rarity,Set,Text,Power,Toughness,Loyalty,Brussels,Nagoya,Phoenix,Total
0,Abrupt Decay,{B}{G},2.0,"[Black, Green]",Instant,Rare,RTR,This spell can't be countered.\nDestroy target...,Not a Creature,Not a Creature,Not a Planeswalker,107,48,112,267
1,Ash Zealot,{R}{R},2.0,[Red],Creature — Human Warrior,Rare,RTR,"First strike, haste\nWhenever a player casts a...",2,2,Not a Planeswalker,4,4,7,15
2,Azorius Charm,{W}{U},2.0,"[Blue, White]",Instant,Uncommon,RTR,Choose one —\n• Creatures you control gain lif...,Not a Creature,Not a Creature,Not a Planeswalker,135,38,53,226
3,Blood Crypt,NaN,0.0,[],Land — Swamp Mountain,Rare,RTR,({T}: Add {B} or {R}.)\nAs Blood Crypt enters ...,Not a Creature,Not a Creature,Not a Planeswalker,42,12,4,58
4,Blood Crypt,NaN,0.0,[],Land — Swamp Mountain,Rare,RNA,({T}: Add {B} or {R}.)\nAs Blood Crypt enters ...,Not a Creature,Not a Creature,Not a Planeswalker,42,12,4,58


In [26]:
def decks_results(results, decklist):
    '''
    Given a dataframe with match results and decklists of the tournament, 
    it transforms to a dataframe of decks results
    
    GW (Games Wins): Total rounds won for each match
    GL (Games Losed): Total rounds losed for each match
    '''
    #transform result table into classification
    results['Result'] = results['Match'].str.split('\s').str.get(0)
    results['GW'] = results['Match'].str.split('\s').str.get(1).str.split('-').str.get(0)
    results['GL'] = results['Match'].str.split('\s').str.get(1).str.split('-').str.get(1)

    #Dict for mapping the player-deck
    player_deck_dict = dict(zip(decklist.Player, decklist.Deck))

    # I was lost without a map... 
    results['Opponent'] = results['Opponent'].map(player_deck_dict)
    results['Player'] = results['Player'].map(player_deck_dict)
    
    return results

In [27]:
brussels_decks_results = decks_results(brussels_matches, brussels_decklist)
brussels_decks = brussels_decks_results.drop('Match', axis=1)
brussels_decks

,Player,Opponent,Result,GW,GL
0,Big Red,NaN,Won,2,0
1,Mono-Black Aggro,Izzet Ensoul,Won,2,0
2,Izzet Ensoul,Izzet Ensoul,Lost,0,2
3,Mono-Red Aggro,Heliod Company,Won,2,1
4,Mono-Green Ramp,Niv to Light,Lost,1,2
...,...,...,...,...,...
1918,Mono-Black Aggro,Azorius Control,Won,2,0
1919,NaN,Mono-White Devotion,Lost,1,2
1920,Orzhov Vampires,Mono-White Devotion,Lost,1,2
1921,Mono-White Devotion,Bant Spirits,Won,2,1


In [28]:
## BRUSSELS DATA

# Which decks has won more matches against which?

brussels_decks[['GW', 'GL']] = brussels_decks[['GW', 'GL']].fillna(0).astype('int32')

# Perform a SELF JOIN on Pandas
brussels_player_group = brussels_decks[['Player', 'Opponent', 'GW', 'GL']].groupby(['Player', 'Opponent']).sum()

brussels_opponent_group = brussels_decks[['Opponent', 'Player', 'GW', 'GL']].groupby(['Opponent', 'Player'], as_index=False).sum().rename(columns={'Player':'Opponent', 'Opponent' : 'Player'})

brussels_total_group = brussels_player_group.merge(brussels_opponent_group, on=['Player', 'Opponent'])
brussels_total_group


,Player,Opponent,GW_x,GL_x,GW_y,GL_y
0,Azorius Control,Azorius Control,14,16,14,16
1,Azorius Control,Azorius Spirits,9,12,16,11
2,Azorius Control,Bant Spirits,2,6,6,1
3,Azorius Control,Big Red,4,0,7,8
4,Azorius Control,Boros Feather,0,2,1,2
...,...,...,...,...,...,...
364,Sultai Delirium,Simic Ramp,4,1,1,2
365,Sultai Delirium,Sultai Delirium,0,2,0,2
366,Sultai Inverter,Mono-Green Ramp,2,1,1,2
367,Temur Delirium,Azorius Control,1,2,2,1


In [29]:
brussels_total_group['Total GW'] = brussels_total_group['GW_x'] + brussels_total_group['GW_y']
brussels_total_group['Total GL'] = brussels_total_group['GL_x'] + brussels_total_group['GL_y']
brussels_total_group['Win Rate'] = brussels_total_group['Total GW'] / (brussels_total_group['Total GW'] + brussels_total_group['Total GL'])
brussels_total_group

,Player,Opponent,GW_x,GL_x,GW_y,GL_y,Total GW,Total GL,Win Rate
0,Azorius Control,Azorius Control,14,16,14,16,28,32,0.466667
1,Azorius Control,Azorius Spirits,9,12,16,11,25,23,0.520833
2,Azorius Control,Bant Spirits,2,6,6,1,8,7,0.533333
3,Azorius Control,Big Red,4,0,7,8,11,8,0.578947
4,Azorius Control,Boros Feather,0,2,1,2,1,4,0.200000
...,...,...,...,...,...,...,...,...,...
364,Sultai Delirium,Simic Ramp,4,1,1,2,5,3,0.625000
365,Sultai Delirium,Sultai Delirium,0,2,0,2,0,4,0.000000
366,Sultai Inverter,Mono-Green Ramp,2,1,1,2,3,3,0.500000
367,Temur Delirium,Azorius Control,1,2,2,1,3,3,0.500000


In [30]:
brussels_total_group = brussels_total_group[['Player', 'Opponent', 'Total GW', 'Total GL', 'Win Rate']]
brussels_total_group

,Player,Opponent,Total GW,Total GL,Win Rate
0,Azorius Control,Azorius Control,28,32,0.466667
1,Azorius Control,Azorius Spirits,25,23,0.520833
2,Azorius Control,Bant Spirits,8,7,0.533333
3,Azorius Control,Big Red,11,8,0.578947
4,Azorius Control,Boros Feather,1,4,0.200000
...,...,...,...,...,...
364,Sultai Delirium,Simic Ramp,5,3,0.625000
365,Sultai Delirium,Sultai Delirium,0,4,0.000000
366,Sultai Inverter,Mono-Green Ramp,3,3,0.500000
367,Temur Delirium,Azorius Control,3,3,0.500000


In [31]:
nagoya_decks_results = decks_results(nagoya_matches, nagoya_decklist)
nagoya_decks = nagoya_decks_results.drop('Match', axis=1)
nagoya_decks


,Player,Opponent,Result,GW,GL
0,Lotus Breach Combo,Mono-Black Aggro,Won,2,1
1,Gb Stompy,NaN,Won,2,0
2,Humans,Big Red,Lost,1,2
3,NaN,NaN,Lost,1,2
4,Izzet Ensoul,Izzet Ensoul,Lost,1,2
...,...,...,...,...,...
937,Mono Red Aggro,NaN,Lost,0,2
938,Big Red,Gb Stompy,Won,2,1
939,NaN,Izzet Ensoul,Lost,1,2
940,NaN,NaN,Won,2,0


In [32]:
## NAGOYA DATA

# Which decks has won more matches against which?

nagoya_decks[['GW', 'GL']] = nagoya_decks[['GW', 'GL']].replace('D',1).fillna(0).astype('int32')

# Perform a SELF JOIN on Pandas
nagoya_player_group = nagoya_decks[['Player', 'Opponent', 'GW', 'GL']].groupby(['Player', 'Opponent']).sum()

nagoya_opponent_group = nagoya_decks[['Opponent', 'Player', 'GW', 'GL']].groupby(['Opponent', 'Player'], as_index=False).sum().rename(columns={'Player':'Opponent', 'Opponent' : 'Player'})

nagoya_total_group = nagoya_player_group.merge(nagoya_opponent_group, on=['Player', 'Opponent'])
nagoya_total_group

,Player,Opponent,GW_x,GL_x,GW_y,GL_y
0,Big Red,Blue Black Inverter,7,14,12,3
1,Big Red,Blue White Control,4,3,2,1
2,Big Red,Blue White Spirits,6,1,2,5
3,Big Red,Gb Stompy,2,1,3,2
4,Big Red,Humans,2,1,1,4
...,...,...,...,...,...,...
108,Niv to Light,Mono-Black Aggro,4,4,2,3
109,Niv to Light,Niv to Light,5,4,5,4
110,Simic Ramp,Blue White Control,0,2,0,2
111,Sram Auras,Blue Black Inverter,4,5,2,3


In [33]:
nagoya_total_group['Total GW'] = nagoya_total_group['GW_x'] + nagoya_total_group['GW_y']
nagoya_total_group['Total GL'] = nagoya_total_group['GL_x'] + nagoya_total_group['GL_y']
nagoya_total_group['Win Rate'] = nagoya_total_group['Total GW'] / (nagoya_total_group['Total GW'] + nagoya_total_group['Total GL'])
nagoya_total_group

,Player,Opponent,GW_x,GL_x,GW_y,GL_y,Total GW,Total GL,Win Rate
0,Big Red,Blue Black Inverter,7,14,12,3,19,17,0.527778
1,Big Red,Blue White Control,4,3,2,1,6,4,0.600000
2,Big Red,Blue White Spirits,6,1,2,5,8,6,0.571429
3,Big Red,Gb Stompy,2,1,3,2,5,3,0.625000
4,Big Red,Humans,2,1,1,4,3,5,0.375000
...,...,...,...,...,...,...,...,...,...
108,Niv to Light,Mono-Black Aggro,4,4,2,3,6,7,0.461538
109,Niv to Light,Niv to Light,5,4,5,4,10,8,0.555556
110,Simic Ramp,Blue White Control,0,2,0,2,0,4,0.000000
111,Sram Auras,Blue Black Inverter,4,5,2,3,6,8,0.428571


In [34]:
nagoya_total_group = nagoya_total_group[['Player', 'Opponent', 'Total GW', 'Total GL', 'Win Rate']]
nagoya_total_group

,Player,Opponent,Total GW,Total GL,Win Rate
0,Big Red,Blue Black Inverter,19,17,0.527778
1,Big Red,Blue White Control,6,4,0.600000
2,Big Red,Blue White Spirits,8,6,0.571429
3,Big Red,Gb Stompy,5,3,0.625000
4,Big Red,Humans,3,5,0.375000
...,...,...,...,...,...
108,Niv to Light,Mono-Black Aggro,6,7,0.461538
109,Niv to Light,Niv to Light,10,8,0.555556
110,Simic Ramp,Blue White Control,0,4,0.000000
111,Sram Auras,Blue Black Inverter,6,8,0.428571


In [35]:
phoenix_decks_results = decks_results(phoenix_matches, phoenix_decklist)
phoenix_decks = nagoya_decks_results.drop('Match', axis=1)
phoenix_decks

,Player,Opponent,Result,GW,GL
0,Lotus Breach Combo,Mono-Black Aggro,Won,2,1
1,Gb Stompy,NaN,Won,2,0
2,Humans,Big Red,Lost,1,2
3,NaN,NaN,Lost,1,2
4,Izzet Ensoul,Izzet Ensoul,Lost,1,2
...,...,...,...,...,...
937,Mono Red Aggro,NaN,Lost,0,2
938,Big Red,Gb Stompy,Won,2,1
939,NaN,Izzet Ensoul,Lost,1,2
940,NaN,NaN,Won,2,0


In [36]:
## PHOENIX DATA

# Which decks has won more matches against which?

phoenix_decks[['GW', 'GL']] = phoenix_decks[['GW', 'GL']].replace('D',1).fillna(0).astype('int32')

# Perform a SELF JOIN on Pandas
phoenix_player_group = phoenix_decks[['Player', 'Opponent', 'GW', 'GL']].groupby(['Player', 'Opponent']).sum()

phoenix_opponent_group = phoenix_decks[['Opponent', 'Player', 'GW', 'GL']].groupby(['Opponent', 'Player'], as_index=False).sum().rename(columns={'Player':'Opponent', 'Opponent' : 'Player'})

phoenix_total_group = phoenix_player_group.merge(phoenix_opponent_group, on=['Player', 'Opponent'])
phoenix_total_group

,Player,Opponent,GW_x,GL_x,GW_y,GL_y
0,Big Red,Blue Black Inverter,7,14,12,3
1,Big Red,Blue White Control,4,3,2,1
2,Big Red,Blue White Spirits,6,1,2,5
3,Big Red,Gb Stompy,2,1,3,2
4,Big Red,Humans,2,1,1,4
...,...,...,...,...,...,...
108,Niv to Light,Mono-Black Aggro,4,4,2,3
109,Niv to Light,Niv to Light,5,4,5,4
110,Simic Ramp,Blue White Control,0,2,0,2
111,Sram Auras,Blue Black Inverter,4,5,2,3


In [37]:
phoenix_total_group['Total GW'] = phoenix_total_group['GW_x'] + phoenix_total_group['GW_y']
phoenix_total_group['Total GL'] = phoenix_total_group['GL_x'] + phoenix_total_group['GL_y']
phoenix_total_group['Win Rate'] = phoenix_total_group['Total GW'] / (phoenix_total_group['Total GW'] + phoenix_total_group['Total GL'])
phoenix_total_group

,Player,Opponent,GW_x,GL_x,GW_y,GL_y,Total GW,Total GL,Win Rate
0,Big Red,Blue Black Inverter,7,14,12,3,19,17,0.527778
1,Big Red,Blue White Control,4,3,2,1,6,4,0.600000
2,Big Red,Blue White Spirits,6,1,2,5,8,6,0.571429
3,Big Red,Gb Stompy,2,1,3,2,5,3,0.625000
4,Big Red,Humans,2,1,1,4,3,5,0.375000
...,...,...,...,...,...,...,...,...,...
108,Niv to Light,Mono-Black Aggro,4,4,2,3,6,7,0.461538
109,Niv to Light,Niv to Light,5,4,5,4,10,8,0.555556
110,Simic Ramp,Blue White Control,0,2,0,2,0,4,0.000000
111,Sram Auras,Blue Black Inverter,4,5,2,3,6,8,0.428571


In [38]:
phoenix_total_group = phoenix_total_group[['Player', 'Opponent', 'Total GW', 'Total GL', 'Win Rate']]
phoenix_total_group

,Player,Opponent,Total GW,Total GL,Win Rate
0,Big Red,Blue Black Inverter,19,17,0.527778
1,Big Red,Blue White Control,6,4,0.600000
2,Big Red,Blue White Spirits,8,6,0.571429
3,Big Red,Gb Stompy,5,3,0.625000
4,Big Red,Humans,3,5,0.375000
...,...,...,...,...,...
108,Niv to Light,Mono-Black Aggro,6,7,0.461538
109,Niv to Light,Niv to Light,10,8,0.555556
110,Simic Ramp,Blue White Control,0,4,0.000000
111,Sram Auras,Blue Black Inverter,6,8,0.428571


## Save the dataframes in csv format

In [41]:
pioneer_sets.to_csv('pioneer_sets.csv', sep='\t')
cards.to_csv('pioneer_cards.csv', sep='\t')

brussels_total_group.to_csv('brussels_decks_results.csv', sep='\t')
nagoya_total_group.to_csv('nagoya_decks_results.csv', sep='\t')
phoenix_total_group.to_csv('phoenix_decks_results.csv', sep='\t')